# Time Series Model on Stock Prices

In [29]:
# Importing Libraries
import pandas as pd
import numpy as np
import itertools
from statistics import mean
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import AutoARIMA
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error
from datetime import date, timedelta
import yfinance as yf

In [30]:
# Getting the date five years ago to download the current timeframe
five_years_ago = (date.today() - timedelta(weeks=260)).strftime("%Y-%m-%d")

# Stocks to analyze
stocks = ['AMD', 'AAL', 'KODK', 'WMT']

# Getting the data for multiple stocks
df = yf.download(stocks, start=five_years_ago)

print("Rows in DataFrame: ", df.shape[0])

[*********************100%***********************]  4 of 4 completed
Rows in DataFrame:  1256


In [31]:
# Storing the dataframes in a dictionary
stock_df = {}

for col in set(df.columns.get_level_values(0)):
    
    # Assigning the information (High, Low, etc.) for each stock in the dictionary
    stock_df[col] = df[col]

# Preprocessing Data

We will use a N day moving average to help smooth out the data and reduce noise, then scale it using a logarithmic scale.

In [32]:
stock_df['LogReturns'] = stock_df['Adj Close'].apply(np.log).diff().dropna()

stock_df['LogClose'] = stock_df['Adj Close'].apply(np.log)

# Visualizing the Data

In [33]:
px.line(stock_df['Adj Close'], 
        x=stock_df['Adj Close'].index, 
        y=stock_df['Adj Close'].columns,
        labels={'variable': 'Stock',
                'value': 'Price'},
        title='Adj Close')


In [34]:
px.line(stock_df['LogClose'], 
        x=stock_df['LogClose'].index, 
        y=stock_df['LogClose'].columns,
        labels={'variable': 'Stock',
                'value': 'Log Price'},
        title='Log of Closing Prices')

## Optimum Parameter Search Function

In [37]:
opt_param = AutoARIMA(start_p=0, start_q=0, 
                      error_action='ignore',
                      suppress_warnings=True)

for stock in tqdm(stocks):

    opt_param.fit(stock_df['LogClose'][stock])

    print(f'Summary for {stock}', '- -'*20)
    display(opt_param.summary())

Summary for AMD


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1256
Model:               SARIMAX(3, 1, 0)   Log Likelihood                2260.123
Date:                Fri, 04 Sep 2020   AIC                          -4510.246
Time:                        17:02:12   BIC                          -4484.572
Sample:                             0   HQIC                         -4500.596
                               - 1256                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0031      0.001      2.686      0.007       0.001       0.005
ar.L1         -0.0630      0.019     -3.230      0.001      -0.101      -0.025
ar.L2          0.0761      0.027      2.800      0.005       0.023       0.129
ar.L3         -0.0604      0.024     -2.556      0.011      -0.107      -0.014
sigma2         0.0016   2.57e-05     62.241      0.000       0.002       0.002
===================================================================================
Ljung-Box (Q):                       40.51   Jarque-Bera (JB):              7732.50
Prob(Q):                              0.45   Prob(JB):                         0.00
Heteroskedasticity (H):               0.65   Skew:                             0.73
Prob(H) (two-sided):                  0.00   Kurtosis:                        15.07
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Summary for AAL


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1256
Model:               SARIMAX(0, 1, 1)   Log Likelihood                2475.070
Date:                Fri, 04 Sep 2020   AIC                          -4946.140
Time:                        17:02:17   BIC                          -4935.870
Sample:                             0   HQIC                         -4942.280
                               - 1256                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1269      0.015      8.210      0.000       0.097       0.157
sigma2         0.0011   1.37e-05     82.547      0.000       0.001       0.001
===================================================================================
Ljung-Box (Q):                       98.03   Jarque-Bera (JB):             21992.55
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               4.84   Skew:                             0.77
Prob(H) (two-sided):                  0.00   Kurtosis:                        23.45
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Summary for KODK


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1256
Model:               SARIMAX(4, 1, 2)   Log Likelihood                1582.870
Date:                Fri, 04 Sep 2020   AIC                          -3151.740
Time:                        17:02:59   BIC                          -3115.795
Sample:                             0   HQIC                         -3138.230
                               - 1256                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.8000      0.147     -5.441      0.000      -1.088      -0.512
ar.L2         -0.2164      0.150     -1.445      0.149      -0.510       0.077
ar.L3         -0.0394      0.053     -0.745      0.456      -0.143       0.064
ar.L4         -0.1743      0.026     -6.633      0.000      -0.226      -0.123
ma.L1          1.1536      0.147      7.860      0.000       0.866       1.441
ma.L2          0.4952      0.181      2.732      0.006       0.140       0.850
sigma2         0.0047   3.91e-05    120.130      0.000       0.005       0.005
===================================================================================
Ljung-Box (Q):                       32.04   Jarque-Bera (JB):            557507.99
Prob(Q):                              0.81   Prob(JB):                         0.00
Heteroskedasticity (H):               8.35   Skew:                             6.58
Prob(H) (two-sided):                  0.00   Kurtosis:                       105.41
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

Summary for WMT


<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1256
Model:               SARIMAX(0, 1, 1)   Log Likelihood                3567.213
Date:                Fri, 04 Sep 2020   AIC                          -7128.425
Time:                        17:03:01   BIC                          -7113.021
Sample:                             0   HQIC                         -7122.635
                               - 1256                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0007      0.000      2.025      0.043    2.35e-05       0.001
ma.L1         -0.1231      0.014     -8.714      0.000      -0.151      -0.095
sigma2         0.0002   2.78e-06     71.463      0.000       0.000       0.000
===================================================================================
Ljung-Box (Q):                       71.31   Jarque-Bera (JB):             12795.12
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.67   Skew:                             0.27
Prob(H) (two-sided):                  0.00   Kurtosis:                        18.63
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

# Using the ARIMA Model
Using the price history from the past N days to make predictions

In [42]:
# Days in the past to train on
days_to_train = 1200 

# Days in the future to predict
days_to_predict = 10

# Establishing a new DFs for predictions
stock_df['Predictions'] = pd.DataFrame(index=stock_df['LogClose'].index,
                                       columns=stock_df['LogClose'].columns)

# Iterate through each stock
for stock in tqdm(stocks):
    
    # Training a model for each day and getting predictions
    for day in tqdm(range(days_to_train, len(stock_df['LogClose'])-1)):

        # Data to use, all days up to the current loop day
        training = stock_df['LogClose'][stock].iloc[:day].dropna()

        # Finding the best parameters
        model    = AutoARIMA(start_p=0, start_q=0,
                             start_P=0, start_Q=0,
                             error_action='ignore',
                             suppress_warnings=True)

        # Getting predictions for the optimum parameters by fitting to the training set            
        forecast = model.fit_predict(training,
                                     n_periods=days_to_predict)


        # Getting the overall average prediction for the next N days
        stock_df['Predictions'][stock].iloc[day] = mean(forecast)

## Preventing Lookahead Bias

In [43]:
# Shifting the predictions up one to prevent lookahead bias
stock_df['Predictions'] = stock_df['Predictions'].shift(1).astype(float)

# Predictions vs Actual Values

In [44]:
# Shift ahead by the predicted amount to compare the actual values to the predictions
pred_df = stock_df['Predictions'].shift(0).apply(np.exp).dropna()

## Plotting the Predictions
Comparing the actual values with the predictions

In [45]:
for stock in stocks:
    
    fig = go.Figure()
    
    # Plotting the actual moving average values
    fig.add_trace(go.Scatter(x=pred_df.index,
                             y=stock_df['Adj Close'][stock].tail(len(pred_df)),
                             name='Actual Adj Close',
                             mode='lines'))
    
    # Plotting the predicted moving average value
    fig.add_trace(go.Scatter(x=pred_df.index,
                             y=pred_df[stock],
                             name='Predicted Adj Close',
                             mode='lines'))
    
    # Setting the labels
    fig.update_layout(title=f'Predicting the Average Adj Close for the Next {days_to_predict} days for {stock}',
                      xaxis_title='Date',
                      yaxis_title='Prices')
    
    fig.show()

## Evaluation Metric

In [47]:
for stock in stocks:
    
    # Finding the root mean squared error
    rmse = mean_squared_error(stock_df['Adj Close'][stock].tail(len(pred_df)),
                              pred_df[stock],
                              squared=False)

    print(f"On average, the model is off by ${round(rmse, 2)} for {stock}\n")

On average, the model is off by $4.08 for AMD

On average, the model is off by $0.82 for AAL

On average, the model is off by $12.97 for KODK

On average, the model is off by $3.05 for WMT



# Trading Signal
Turning the model into a Trading Signal

In [48]:
def position_decision(num, thres=3):
    """
    Computes the position based on the threshold assumed as percentage
    If number exceeds the threshold, then 1 or -1 is assigned, 0 if otherwise
    """
    if num > thres/100:
        return -1
    
    elif num < -1*(thres/100):
        return 1
    
    else:
        return 0

### Creating a Trading DF
with relevant columns to evaluate model performance

In [49]:
# Creating a DF for trading the model
trade_df = {}

# Transforming values back to normal and dropping nans
trade_df['Predictions'] = stock_df['Predictions'].apply(np.exp).dropna()

# Getting Log Returns
trade_df['LogReturns'] = stock_df['LogReturns'].tail(len(trade_df['Predictions'])+1)

# Percentage difference between current closing price and the forecasted average Adj Close in N days
trade_df['Forecast'] = (trade_df['Predictions'] / stock_df['Adj Close'].tail(len(trade_df['Predictions']))) - 1

# Getting positions
trade_df['Positions'] = pd.DataFrame(index=trade_df['Forecast'].index,
                                     columns=trade_df['Forecast'].columns)

for stock in stocks:
    
    trade_df['Positions'][stock] = trade_df['Forecast'][stock].apply(lambda x: position_decision(x, thres=3)/len(stocks))

display(trade_df['Forecast'])
display(trade_df['Positions'])

,AAL,AMD,KODK,WMT
Date,,,,
2020-06-19,0.060237,0.019876,-0.016233,-0.003011
2020-06-22,0.090826,0.002819,-0.015903,-0.025935
2020-06-23,0.138458,0.019568,0.079235,-0.008486
2020-06-24,0.160119,0.061775,0.011187,0.012864
2020-06-25,0.089965,0.056261,-0.111063,0.015250
2020-06-26,0.084848,0.062032,-0.038572,0.021101
2020-06-29,0.017385,0.049376,0.028943,0.009625
2020-06-30,-0.040096,-0.030198,0.026475,-0.007334
2020-07-01,0.049365,-0.029564,0.058615,-0.002478


,AAL,AMD,KODK,WMT
Date,,,,
2020-06-19,-0.25,0.00,0.00,0.00
2020-06-22,-0.25,0.00,0.00,0.00
2020-06-23,-0.25,0.00,-0.25,0.00
2020-06-24,-0.25,-0.25,0.00,0.00
2020-06-25,-0.25,-0.25,0.25,0.00
2020-06-26,-0.25,-0.25,0.25,0.00
2020-06-29,0.00,-0.25,0.00,0.00
2020-06-30,0.25,0.25,0.00,0.00
2020-07-01,-0.25,0.00,-0.25,0.00


## Plotting the Positions and Predicted Moves

In [50]:
# Plotting positions over time
fig = px.line(trade_df['Positions'], 
              x=trade_df['Positions'].index, 
              y=stocks,
              title='Positions over Time')

fig.show()

# Getting the number of positions
pos = trade_df['Positions'].apply(pd.value_counts)

# Plotting total positions
fig1 = px.bar(pos, 
              x=pos.index, 
              y=pos.columns,
              title='Total Positions',
              labels={'variable':'Stocks',
                      'value':'Count of Positions',
                      'index':'Position'})

fig1.show()

# Calculating and Plotting the Potential Returns
With the ARMA model

## Returns on Each Individual Stock

In [51]:
# Positions will need to be shifted up by 1 to remove look ahead bias
returns = trade_df['Positions'] * trade_df['LogReturns']

# Calculating the performance as we take the cumulative sum of the returns and transform the values back to normal
performance = returns.cumsum().apply(np.exp)

# Plotting the performance per stock
px.line(performance,
        x=performance.index,
        y=performance.columns,
        title='Returns Per Stock Using ARIMA Forecast',
        labels={'variable':'Stocks',
                'value':'Returns'})

## Returns on the Overall Portfolio

In [52]:
# Returns for the portfolio
returns = (trade_df['Positions'] * trade_df['LogReturns']).sum(axis=1)

# Returns for SPY
spy = yf.download('SPY', start=returns.index[0])

spy = spy['Adj Close'].apply(np.log).diff().dropna().cumsum().apply(np.exp)

# Calculating the performance as we take the cumulative sum of the returns and transform the values back to normal
performance = returns.cumsum().apply(np.exp)


# Plotting the comparison between SPY returns and GARCH returns
fig = go.Figure()

fig.add_trace(go.Scatter(x=spy.index,
                         y=spy,
                         name='SPY Returns',
                         mode='lines'))

fig.add_trace(go.Scatter(x=performance.index,
                         y=performance.values,
                         name='ARIMA Returns on Portfolio',
                         mode='lines'))

fig.update_layout(title='SPY vs ARIMA Overall Portfolio Returns',
                  xaxis_title='Date',
                  yaxis_title='% Change')

fig.show()

[*********************100%***********************]  1 of 1 completed
